# 1D Convolution on CPU

## 2. Install TVM

In [1]:
!pip install tlcpack-nightly-cu102 -f https://tlcpack.ai/wheels

Looking in links: https://tlcpack.ai/wheels


In [2]:
!pip install "numpy<2.0.0"

## 3. Implement `make_conv1d_cpu_scheduler_func` function in `src.ops`

In that function, you are required to implemented 1D convolution and use TVM to optimize it.
Let $x \in \mathbb{R}^m$ and $y \in \mathbb{R}^n$, then
$$
\operatorname{conv1d}(x, y)_i = \sum_{j=-\infty}^{\infty} x[j]y[i-j], \forall i \in \{0, 1, \dots, m + n - 1\}
$$

Please use zero padding and unit stride. Please see the numpy convolution function for more detail: [link](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html).

The `make_conv1d_cpu_scheduler_func` takes $m$ and $n$, which are the size of the two 1D input array.
You should return both the TVM schedule and the TVM operator for
1. Input $x$
2. Input $y$
3. Output $out$

The schedule should be able to used to build a function with signature $func(x, y, out)$.
Please see the following cells the usage.

In [22]:
import os
import tvm
import numpy as np
from tvm import te, autotvm
import time
import torch
import torch.nn.functional as F

# naive baseline
def make_conv1d_cpu_scheduler_naive(M, N):
    A = te.placeholder((M,), name="A")  # input tensor placeholder
    W = te.placeholder((N,), name="W")  # weight tensor placeholder

    k = te.reduce_axis((0, M + N - 1), "k")   # k in [0, M+N-1)
    B = te.compute(
        (M + N - 1,),   # output shape, n from (0, M + N - 1)
        # if_then_else: if satisfy "any" condition, return 0 else A[k] * W[n - k]
        lambda n: te.sum(tvm.tir.if_then_else(
            tvm.tir.any(k < 0, k >= M, n - k < 0, n - k >= N),
            tvm.tir.const(0.0, "float32"),
            A[k] * W[n - k]), axis=k),
        name="B",
    )
    s = te.create_schedule(B.op)
    print("=" * 100)
    print(tvm.lower(s, [A, W, B], simple_mode=True))
    print("=" * 100)

    return s, A, W, B

# optimize v0: shrink the range of k to reduce if else
def make_conv1d_cpu_scheduler_v0(M, N, verbose=True):
    A = te.placeholder((M,), name="A", dtype="float32")
    W = te.placeholder((N,), name="W", dtype="float32")

    k = te.reduce_axis((0, M), "k")   # k in [0, M)
    B = te.compute(
        (M + N - 1,),
        lambda n: te.sum(tvm.tir.if_then_else(
            tvm.tir.any(k < 0, k >= M, n - k < 0, n - k >= N),
            tvm.tir.const(0.0, "float32"),
            A[k] * W[n - k]), axis=k),
        name="B",
    )

    s = te.create_schedule(B.op)
    if verbose:
        print("=" * 100)
        print(tvm.lower(s, [A, W, B], simple_mode=True))
        print("=" * 100)
    return s, A, W, B

# optimize v1: v0 + parallel
def make_conv1d_cpu_scheduler_v1(M, N, verbose=True):
    s, A, W, B = make_conv1d_cpu_scheduler_v0(M, N, False)
    n_axis = B.op.axis[0]   # output axis
    s[B].parallel(n_axis)   # parallel for output axis
    if verbose:
        print("=" * 100)
        print(tvm.lower(s, [A, W, B], simple_mode=True))
        print("=" * 100)
    return s, A, W, B

# optimize v2: v0 + parallel + split + vectorize
def make_conv1d_cpu_scheduler_v2(M, N, factor=8, verbose=True):
    s, A, W, B = make_conv1d_cpu_scheduler_v0(M, N, False)
    n_axis = B.op.axis[0]
    # AVX2, bw=256 for vectorization. 8 * float32 or 16 * float16
    outer, inner = s[B].split(n_axis, factor=factor)
    s[B].parallel(outer)
    s[B].vectorize(inner)   # CPU SIMD usage
    if verbose:
        print("=" * 100)
        print(tvm.lower(s, [A, W, B], simple_mode=True))
        print("=" * 100)
    return s, A, W, B

# optimize v3: v2 + k_axis split + unroll
def make_conv1d_cpu_scheduler_v3(M, N, factor=8, verbose=True):
    s, A, W, B = make_conv1d_cpu_scheduler_v2(M, N, factor, False)

    k_axis = B.op.reduce_axis[0]
    k_outer, k_inner = s[B].split(k_axis, factor=factor)
    s[B].unroll(k_inner)  # unroll to reduce loop overhead
    if verbose:
        print("=" * 100)
        print(tvm.lower(s, [A, W, B], simple_mode=True))
        print("=" * 100)
    return s, A, W, B

# optimize v4: compute refactor(minimize if-else) + parallel + split + vectorize
def make_conv1d_cpu_scheduler_v4(M, N, factor=8, verbose=True):
    A = te.placeholder((M,), name="A", dtype="float32")
    W = te.placeholder((N,), name="W", dtype="float32")
    k = te.reduce_axis((0, N), name="k")

    B = te.compute(
        (M + N - 1,),
        lambda n: te.sum(
            tvm.tir.if_then_else(
                tvm.tir.all(n - k >= 0, n - k < M),
                A[n - k] * W[k],
                tvm.tir.const(0.0, "float32")
            ),
            axis=k
        ),
        name="B"
    )
    s = te.create_schedule(B.op)
    n_axis = B.op.axis[0]
    outer, inner = s[B].split(n_axis, factor=factor)
    s[B].parallel(outer)
    s[B].vectorize(inner)   # CPU SIMD usage
    if verbose:
        print("=" * 100)
        print(tvm.lower(s, [A, W, B], simple_mode=True))
        print("=" * 100)
    return s, A, W, B


# benchmark for tvm implementation
def benchmark_conv1d_tvm(schedule_func, M, N, device, a_np, w_np, num_runs=30, repeat=20):
    s, A, W, B = schedule_func(M, N)
    func = tvm.build(s, [A, W, B], target="llvm")

    a_tvm = tvm.nd.array(a_np, device)
    w_tvm = tvm.nd.array(w_np, device)
    out_tvm = tvm.nd.array(np.zeros((M + N - 1,), dtype=a_np.dtype), device)

    evaluator = func.time_evaluator(func.entry_name, device, number=num_runs, repeat=repeat)
    cost = evaluator(a_tvm, w_tvm, out_tvm).mean  # average time in seconds
    return cost, out_tvm.asnumpy(), func, (s, A, W, B)

# benchmark for numpy
def benchmark_conv1d_numpy(a_np, w_np, num_runs=10):
    t0 = time.time()
    out = None
    for _ in range(num_runs):
        out = np.convolve(a_np, w_np)
    t1 = time.time()
    return (t1 - t0) / num_runs, out




In [6]:
M = 4096
N = 128
dtype = "float32"
np.random.seed(0)
a_np = np.random.rand(M).astype(dtype)
w_np = np.random.rand(N).astype(dtype)
ref = np.convolve(a_np, w_np)

# naive TVM
dev = tvm.cpu()
naive_time, naive_res, naive_func, naive_comp = benchmark_conv1d_tvm(
    make_conv1d_cpu_scheduler_naive, M, N, dev, a_np, w_np
)
np.testing.assert_allclose(naive_res, ref, rtol=1e-4)
print(f"[TVM Naive] time: {naive_time*1e3:.4f} ms")


# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((4096,), "float32"), W: T.Buffer((128,), "float32"), B: T.Buffer((4223,), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        for n in range(4223):
            B[n] = T.float32(0)
            for k in range(4223):
                cse_var_1: T.int32 = n - k
                B[n] = B[n] + T.if_then_else(4096 <= k or cse_var_1 < 0 or 128 <= cse_var_1, T.float32(0), A[k] * W[cse_var_1])
[TVM Naive] time: 24.3525 ms


In [26]:
# optimized TVM v0
opt_time, opt_res, opt_func, opt_comp = benchmark_conv1d_tvm(
    make_conv1d_cpu_scheduler_v0, M, N, dev, a_np, w_np
)
np.testing.assert_allclose(opt_res, ref, rtol=1e-4)
print(f"[TVM Manual Opt v0] time: {opt_time*1e3:.4f} ms")


# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((4096,), "float32"), W: T.Buffer((128,), "float32"), B: T.Buffer((4223,), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        for n in range(4223):
            B[n] = T.float32(0)
            for k in range(4096):
                cse_var_1: T.int32 = n - k
                B[n] = B[n] + T.if_then_else(cse_var_1 < 0 or 128 <= cse_var_1, T.float32(0), A[k] * W[cse_var_1])
[TVM Manual Opt v0] time: 23.0471 ms


In [24]:
# optimized TVM v1
opt_time, opt_res, opt_func, opt_comp = benchmark_conv1d_tvm(
    make_conv1d_cpu_scheduler_v1, M, N, dev, a_np, w_np
)
np.testing.assert_allclose(opt_res, ref, rtol=1e-4)
print(f"[TVM Manual Opt v1] time: {opt_time*1e3:.4f} ms")


# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((4096,), "float32"), W: T.Buffer((128,), "float32"), B: T.Buffer((4223,), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        for n in T.parallel(4223):
            B[n] = T.float32(0)
            for k in range(4096):
                cse_var_1: T.int32 = n - k
                B[n] = B[n] + T.if_then_else(cse_var_1 < 0 or 128 <= cse_var_1, T.float32(0), A[k] * W[cse_var_1])
[TVM Manual Opt v1] time: 22.9158 ms


In [25]:
# optimized TVM v2
opt_time, opt_res, opt_func, opt_comp = benchmark_conv1d_tvm(
    make_conv1d_cpu_scheduler_v2, M, N, dev, a_np, w_np
)
np.testing.assert_allclose(opt_res, ref, rtol=1e-4)
print(f"[TVM Manual Opt v2] time: {opt_time*1e3:.4f} ms")


# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((4096,), "float32"), W: T.Buffer((128,), "float32"), B: T.Buffer((4223,), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        for n_outer in T.parallel(528):
            for n_inner_s in range(8):
                if T.likely(n_outer * 8 + n_inner_s < 4223):
                    B[n_outer * 8 + n_inner_s] = T.float32(0)
            for k, n_inner_s in T.grid(4096, 8):
                if T.likely(n_outer * 8 + n_inner_s < 4223):
                    cse_var_2: T.int32 = n_outer * 8 + n_inner_s
                    cse_var_1: T.int32 = cse_var_2 - k
                    B[cse_var_2] = B[cse_var_2] + T.if_then_else(cse_var_1 < 0 or 128 <= cse_var_1, T.float32(0), A[k] * W[cse_var_1])
[TVM Manual Opt v2] time: 16.0384 ms


In [27]:
# optimized TVM v3
opt_time, opt_res, opt_func, opt_comp = benchmark_conv1d_tvm(
    make_conv1d_cpu_scheduler_v3, M, N, dev, a_np, w_np
)
np.testing.assert_allclose(opt_res, ref, rtol=1e-4)
print(f"[TVM Manual Opt v3] time: {opt_time*1e3:.4f} ms")

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((4096,), "float32"), W: T.Buffer((128,), "float32"), B: T.Buffer((4223,), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        for n_outer in T.parallel(528):
            for n_inner_s in range(8):
                if T.likely(n_outer * 8 + n_inner_s < 4223):
                    B[n_outer * 8 + n_inner_s] = T.float32(0)
            for k_outer in range(512):
                for n_inner_s in range(8):
                    if T.likely(n_outer * 8 + n_inner_s < 4223):
                        cse_var_3: T.int32 = k_outer * 8
                        cse_var_2: T.int32 = n_outer * 8 + n_inner_s
                        cse_var_1: T.int32 = cse_var_2 - cse_var_3
                        B[cse_var_2] = B[cse_var_2] + T.if_then_else(n_outer - k_outer < 0 or 128 <= cse_var_1, T.float32(0), A[cse_var_3] * W

In [28]:
# optimized TVM v4
opt_time, opt_res, opt_func, opt_comp = benchmark_conv1d_tvm(
    make_conv1d_cpu_scheduler_v4, M, N, dev, a_np, w_np
)
np.testing.assert_allclose(opt_res, ref, rtol=1e-4)
print(f"[TVM Manual Opt v4] time: {opt_time*1e3:.4f} ms")



# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((4096,), "float32"), W: T.Buffer((128,), "float32"), B: T.Buffer((4223,), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        for n_outer in T.parallel(528):
            for n_inner_s in range(8):
                if T.likely(n_outer * 8 + n_inner_s < 4223):
                    B[n_outer * 8 + n_inner_s] = T.float32(0)
            for k, n_inner_s in T.grid(128, 8):
                if T.likely(n_outer * 8 + n_inner_s < 4223):
                    cse_var_2: T.int32 = n_outer * 8 + n_inner_s
                    cse_var_1: T.int32 = cse_var_2 - k
                    B[cse_var_2] = B[cse_var_2] + T.if_then_else(0 <= cse_var_1 and cse_var_1 < 4096, A[cse_var_1] * W[k], T.float32(0))
[TVM Manual Opt v4] time: 0.5661 ms


In [29]:
# numPy baseline
numpy_time, numpy_out = benchmark_conv1d_numpy(a_np, w_np, num_runs=10)
print(f"[NumPy]   time: {numpy_time*1e3:.4f} ms")
np.testing.assert_allclose(numpy_out, ref, rtol=1e-4)


[NumPy]   time: 0.2140 ms


In [14]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.00GHz
    CPU family:           6
    Model:                85
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             3
    BogoMIPS:             4000.22
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 cl
                          flush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc re
                          p_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3
                           fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand
                           hypervisor lahf_lm abm 3dnowprefetch i

In [15]:
from tvm import autotvm

@autotvm.template("tutorial/conv1d_auto_tune")
def conv1d_auto_tune(M, N):
    A = te.placeholder((M,), name="A", dtype="float32")
    W = te.placeholder((N,), name="W", dtype="float32")
    k = te.reduce_axis((0, N), name="k")

    B = te.compute(
        (M + N - 1,),
        lambda i: te.sum(
            tvm.tir.if_then_else(
                tvm.tir.all(i - k >= 0, i - k < M),
                A[i - k] * W[k],
                tvm.tir.const(0.0, "float32")
            ),
            axis=k
        ),
        name="B"
    )

    s = te.create_schedule(B.op)
    cfg = autotvm.get_config()
    i = B.op.axis[0]
    k_ = B.op.reduce_axis[0]

    # define search space
    cfg.define_split("tile_i", i, num_outputs=2, filter=lambda x: x.size[-1] <= 64)
    cfg.define_split("tile_k", k_, num_outputs=2, filter=lambda x: x.size[-1] <= 64)
    cfg.define_knob("vectorize", [True, False])
    cfg.define_knob("unroll", [0, 8, 16])

    # schedule according to config
    A_local = s.cache_read(A, "local", [B])
    W_local = s.cache_read(W, "local", [B])

    i_outer, i_inner = cfg["tile_i"].apply(s, B, i)
    k_outer, k_inner = cfg["tile_k"].apply(s, B, k_)

    s[B].parallel(i_outer)

    if cfg["vectorize"].val:
        s[B].vectorize(i_inner)

    unroll_factor = cfg["unroll"].val
    if unroll_factor > 0:
        s[B].unroll(i_inner)

    s[A_local].compute_at(s[B], i_outer)
    s[W_local].compute_at(s[B], i_outer)

    return s, [A, W, B]


def tune_conv1d(M, N, trials=200, log_file="conv1d.log"):
    task = autotvm.task.create("tutorial/conv1d_auto_tune", args=(M, N), target="llvm")
    print("AutoTVM Task:", task.name, "Search Space Size:", len(task.config_space))

    measure_option = autotvm.measure_option(
        builder=autotvm.LocalBuilder(),
        runner=autotvm.LocalRunner(number=5, repeat=1, min_repeat_ms=100)
    )

    tuner = autotvm.tuner.XGBTuner(task)
    tuner.tune(
        n_trial=trials,
        measure_option=measure_option,
        callbacks=[
            autotvm.callback.log_to_file(log_file),
            autotvm.callback.progress_bar(trials)
        ]
    )

    # build kernel from the best history
    with autotvm.apply_history_best(log_file):
        with tvm.target.Target("llvm"):
            s, arg_bufs = conv1d_auto_tune(M, N)
            func = tvm.build(s, arg_bufs, target="llvm")

    return func

def benchmark_conv1d_autotvm(M, N, dev, a_np, w_np, trials=200, num_runs=10):
    func = tune_conv1d(M, N, trials=trials)
    a_tvm = tvm.nd.array(a_np, dev)
    w_tvm = tvm.nd.array(w_np, dev)
    out_tvm = tvm.nd.array(np.zeros((M + N - 1,), dtype=a_np.dtype), dev)

    evaluator = func.time_evaluator(func.entry_name, dev, number=num_runs, repeat=1)
    cost = evaluator(a_tvm, w_tvm, out_tvm).mean
    return cost, out_tvm.asnumpy()


auto_time, auto_res = benchmark_conv1d_autotvm(
    M, N, dev, a_np, w_np, trials=50, num_runs=10
)
np.testing.assert_allclose(auto_res, ref, rtol=1e-4)
print(f"[TVM AutoTVM] time: {auto_time*1e3:.4f} ms")


AutoTVM Task: tutorial/conv1d_auto_tune Search Space Size: 84
 Current/Best:    5.45/   5.89 GFLOPS | Progress: (50/50) | 89.83 s Done.
[TVM AutoTVM] time: 0.7022 ms


In [16]:
# pytest
%pip install ipytest
import ipytest
ipytest.autoconfig()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.5 MB/s eta 0:00:00


In [17]:
%%ipytest

import tvm
import torch
import pytest
import timeit
import numpy as np
import torch.nn.functional as F


dev = tvm.device('llvm', 0)


def make_conv1d_cpu_func(M, N):
    s, A, W, O = make_conv1d_cpu_scheduler_v4(M, N)
    func = tvm.build(s, [A, W, O], "llvm")
    return func


def ans_np(a_np, w_np):
    a_np = a_np.flatten()
    w_np = w_np.flatten()
    return np.convolve(a_np, w_np)


def ans_torch(a_torch, w_torch):
    M, N = a_torch.size(0), w_torch.size(0)
    b_torch = F.conv1d(a_torch, w_torch, bias=None, stride=1,
                       padding=(N - 1), dilation=1, groups=1)
    return b_torch


@pytest.mark.parametrize('execution_number', range(5))
def test1_M1_N1(execution_number):
    # Define dimension
    M = 1
    N = 1
    func = make_conv1d_cpu_func(M, N)

    # Create random test data
    np.random.seed(seed=execution_number)
    a_np = np.random.rand(M).astype(np.float32)
    w_np = np.random.rand(N).astype(np.float32)
    b_np = ans_np(a_np, w_np)

    a = tvm.nd.array(a_np, dev)
    w = tvm.nd.array(w_np, dev)
    b = tvm.nd.array(np.zeros((M + N - 1), dtype='float32'), dev)
    func(a, w, b)
    b_out = b.numpy()

    assert b_np.shape == b_out.shape, \
        "Shape mismatch: " + str(b_np.shape) + "\t" + str(b_out.shape)
    assert np.allclose(b_np, b_out), "Value mismatch: %s %s" % (b_np, b_out)


@pytest.mark.parametrize('execution_number', [1, 10, 100, 1000, 10000])
def test1_Mvar_N1024(execution_number):
    # Define dimension
    M = execution_number
    N = 1024
    func = make_conv1d_cpu_func(M, N)

    # Create random test data
    np.random.seed(seed=1024)
    a_np = np.random.rand(M).astype(np.float32)
    w_np = np.random.rand(N).astype(np.float32)
    b_np = ans_np(a_np, w_np)

    a = tvm.nd.array(a_np, dev)
    w = tvm.nd.array(w_np, dev)
    b = tvm.nd.array(np.zeros((M + N - 1), dtype='float32'), dev)
    func(a, w, b)
    b_out = b.numpy()

    assert b_np.shape == b_out.shape, \
        "Shape mismatch: " + str(b_np.shape) + "\t" + str(b_out.shape)
    assert np.allclose(b_np, b_out), "Value mismatch: %s %s" % (b_np, b_out)


@pytest.mark.parametrize('execution_number', [1, 10, 100, 1000, 10000])
def test1_M1024_Nvar(execution_number):
    # Define dimension
    M = 1024
    N = execution_number
    func = make_conv1d_cpu_func(M, N)

    # Create random test data
    np.random.seed(seed=1024)
    a_np = np.random.rand(M).astype(np.float32)
    w_np = np.random.rand(N).astype(np.float32)
    b_np = ans_np(a_np, w_np)

    a = tvm.nd.array(a_np, dev)
    w = tvm.nd.array(w_np, dev)
    b = tvm.nd.array(np.zeros((M + N - 1), dtype='float32'), dev)
    func(a, w, b)
    b_out = b.numpy()

    assert b_np.shape == b_out.shape, \
        "Shape mismatch: " + str(b_np.shape) + "\t" + str(b_out.shape)
    assert np.allclose(b_np, b_out), "Value mismatch: %s %s" % (b_np, b_out)

...............                                                                              [100%]
15 passed in 4.20s
